# Получение данных об инфраструктуре в радиусе 1 км от квартиры

## Импорт библиотек

In [50]:
import requests
import json
import time
import pandas as pd

## Открытие данных

In [44]:
df = pd.read_csv('../clean_categorizated_data_without_outliers_with_coordinates.csv', sep=';')

In [45]:
df.sample(3)

,number of rooms,area of apartment,number of floors,apartment floor,price,year of construction,elevator,full address,concierge,garbage chute,...,district_Ленинский,district_Мотовилихинский,district_Орджоникидзевский,district_Свердловский,parking_за шлагбаумом во дворе,parking_открытая во дворе,parking_наземная многоуровневая,parking_подземная,lat,lon
1644,3,80.6,16,5,4990000.0,2013,1.0,"Пермский край, Пермь, ул. Докучаева, 52",0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.033818,56.141256
1185,2,55.7,16,2,4900000.0,2017,1.0,"Пермский край, Пермь, ул. Овчинникова, 35А",0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.994669,56.218867
1111,1,41.0,22,13,4300000.0,2015,1.0,"Пермский край, Пермь, Подлесная ул., 43А",1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.996586,56.163601


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1767 entries, 0 to 1766
Data columns (total 38 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   number of rooms                  1767 non-null   int64  
 1   area of apartment                1767 non-null   float64
 2   number of floors                 1767 non-null   int64  
 3   apartment floor                  1767 non-null   int64  
 4   price                            1767 non-null   float64
 5   year of construction             1767 non-null   int64  
 6   elevator                         1767 non-null   float64
 7   full address                     1767 non-null   object 
 8   concierge                        1767 non-null   int64  
 9   garbage chute                    1767 non-null   int64  
 10  repair_дизайнерский              1767 non-null   float64
 11  repair_евро                      1767 non-null   float64
 12  repair_косметический

## Парсинг данных

In [54]:
all_categories = {
    "eat":{
        "amenities": ["cafe", "fast_food", "pub", "restaurant"],
        "data": set()
    },
    "edu":{
        "amenities": ["college", "kindergarten", "library", "school", "university"],
        "data": set()
    },
    "health":{
        "amenities": ["clinic", "hospital"],
        "data": set()
    },
    "culture":{
        "amenities": ["cinema", "fountain", "theatre"],
        "data": set()
    },
}

In [55]:
overpass_url = "http://overpass-api.de/api/interpreter"

for category in all_categories:
    for amenity in all_categories[category]['amenities']:
        time.sleep(3)
        overpass_query = f"""
        [out:json];
        node(57.5,55,58.5,56.5)[amenity={amenity}];out;
        """
        response = requests.get(overpass_url, 
                                params={'data': overpass_query})
        try:
            data = response.json()
            for element in data['elements']:
              if element['type'] == 'node':
                lon = element['lon']
                lat = element['lat']
                all_categories[category]['data'].add((lat, lon))
        except Exception as ex:
            print(amenity)
            print(ex)

In [56]:
for key in all_categories:
    print(key + ": " + str(len(all_categories[key]['data'])))

eat: 554
edu: 137
health: 86
culture: 20


In [58]:
for key in all_categories:
    all_categories[key]['data'] = list(all_categories[key]['data'])

In [61]:
with open("../amenities.json", 'w') as f:
    json.dump(all_categories, f)